In [ ]:
import altair as alt
from vega_datasets import data

# Data generators for the background
sphere = alt.Chart(alt.sphere()).mark_geoshape(fill="#9ecae9", opacity=0.4)

graticule = alt.Chart(alt.graticule(step=(180, 0))).mark_geoshape(
    stroke="#54a24b", strokeWidth=0.8
)

# Source of land data
countries = alt.Chart(alt.topo_feature(data.world_110m.url, "countries")).mark_geoshape(
    fill="#bab0ac", stroke="white", opacity=0.4
)


# Layering and configuring the components
map0 = (
    alt.layer(sphere, graticule, countries)
    .project("orthographic", rotate=(-30, -30, 0))
    .properties(width=400, height=400)
    .configure_view(stroke=None)
)
map0

In [ ]:
map1 = (
    alt.layer(sphere, graticule, countries)
    .project("conicConformal", rotate=(-10, -52, 0), parallels=(35, 65), scale=450)
    .properties(width=400, height=400)
)
map1

In [ ]:
from geopandas import GeoDataFrame

github_url = "https://raw.githubusercontent.com/{user}/{repo}/master/{path}"

gb_lad = GeoDataFrame.from_file(
    github_url.format(
        user="martinjc",
        repo="UK-GeoJSON",
        path="json/administrative/gb/lad.json",
    )
)

In [ ]:
map2 = (
    alt.layer(
        sphere,
        alt.Chart(gb_lad).mark_geoshape(fill="#bab0ac", stroke="white", opacity=0.4),
        graticule.mark_geoshape(stroke="#54a24b", strokeWidth=3.2),
    )
    .project("transverseMercator", rotate=(2, -54.5, 0), scale=2200 * 1270 // 400)
    .properties(
        width=1270, height=1270
    )  # svg export fails, so changing dpi with gimp => image/print size
    .configure_view(stroke=None)
)
map2

In [ ]:
import altair as alt
from vega_datasets import data

boroughs = alt.Chart(alt.topo_feature(data.londonBoroughs.url, "boroughs"))


map3 = (
    alt.layer(
        boroughs.mark_geoshape(
            fill="#bab0ac", stroke="white", opacity=0.4
        ).transform_filter('datum.id != "Greenwich"'),
        boroughs.mark_geoshape(
            fill="#54a24b", stroke="white", opacity=0.4
        ).transform_filter('datum.id == "Greenwich"'),
        graticule,
    )
    .project("transverseMercator", rotate=(0.1, -51.5, 0), scale=40000)  # (-2, -49, 0),
    .properties(width=400, height=400)
    .configure_view(stroke=None)
)
map3

In [ ]:
from cartes.osm import Nominatim, Overpass
from cartes.osm.overpass.core import Relation

greenwich = Overpass.request(bounds=(-0.0255, 51.4630, 0.0188, 51.4860), nwr=dict())


class Route(Relation):
    pass


class Public_transport(Relation):
    pass


class AssociatedStreet(Relation):
    pass


class Site(Relation):
    pass


class Restriction(Relation):
    pass


Relation.subclasses["associatedStreet"] = Relation.subclasses["associatedstreet"]
Relation.subclasses["route_master"] = Relation.subclasses["associatedstreet"]

In [ ]:
map4 = (
    alt.layer(
        alt.Chart(
            greenwich.query(
                'leisure.notnull() and leisure == "park" and type != "node"'
            )
        ).mark_geoshape(fill="#54a24b", stroke="#bab0ac", opacity=0.4),
        alt.Chart(greenwich.query("building.notnull()")).mark_geoshape(
            fill="#bab0ac", stroke="#bab0ac"
        ),
        alt.Chart(greenwich.query("highway.notnull()")).mark_geoshape(
            fill="none", stroke="#bab0ac", opacity=0.4
        ),
        alt.Chart(
            greenwich.query('natural.notnull() and natural=="water"')
        ).mark_geoshape(fill="#9ecae9", stroke="#9ecae9"),
        graticule,
    )
    .project(  # OSGB adjusted
        "transverseMercator", rotate=(0, -51.475, 0), scale=1000000
    )
    .properties(width=400, height=400)
    .configure_view(stroke=None)
    .configure_text(
        font="Fontin Sans", color="#54a24b", fontSize=14, fontStyle="italic"
    )
)

map4

In [ ]:
def text(name):
    return alt.Chart(pd.DataFrame.from_records([{"name": name}])).encode(
        alt.Text("name:N")
    )


map5 = (
    alt.layer(
        text("[grɛ́nɪʧ]").mark_text(
            font="Noto Serif", color="#54a24b", fontSize=28 * 4, dy=50 * 4
        ),
        text("Greenwich").mark_text(font="Fira", color="#54a24b", fontSize=36 * 4),
    )
    .properties(width=1270, height=1270)
    .configure_view(stroke=None)
)
map5